## Libraries 

In [1]:
from googlesearch import search # get zpid
import pandas as pd
import requests
import json
import time
import io
import plotly.express as px
import numpy as np

api_key = "API KEY FOR SCRAPEAK" # Enter your API key from ScrapPeak https://www.scrapeak.com

import gspread
import gspread_dataframe as gd
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials
from oauth2client.service_account import ServiceAccountCredentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

## Inital Dataframe
Dict Keys == town names

Dict Values == link to listings

Links should be include all filters (ie price range, beds, baths, house types, etc )

In [2]:
# On https://www.zillow.com apply the filters and search the town then copy and paste the link into the dictionary below
data_dict = {"Glastonbury":"https://www.zillow.com/glastonbury-ct/houses/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22Glastonbury%2C%20CT%22%2C%22mapBounds%22%3A%7B%22west%22%3A-72.75130331103514%2C%22east%22%3A-72.34961568896483%2C%22south%22%3A41.58696978293469%2C%22north%22%3A41.79053737118673%7D%2C%22mapZoom%22%3A12%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A8509%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22price%22%3A%7B%22min%22%3A300000%2C%22max%22%3A650000%7D%2C%22con%22%3A%7B%22value%22%3Afalse%7D%2C%22apa%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22mp%22%3A%7B%22min%22%3A1473%2C%22max%22%3A4014%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22tow%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22apco%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%7D",
        "Simsbury":"https://www.zillow.com/simsbury-ct/houses/?searchQueryState=%7B%22usersSearchTerm%22%3A%22Glastonbury%2C%20CT%22%2C%22mapBounds%22%3A%7B%22north%22%3A41.96968714948398%2C%22east%22%3A-72.61671018896482%2C%22south%22%3A41.766688340835344%2C%22west%22%3A-73.01839781103513%7D%2C%22mapZoom%22%3A12%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22price%22%3A%7B%22min%22%3A300000%2C%22max%22%3A650000%7D%2C%22con%22%3A%7B%22value%22%3Afalse%7D%2C%22apa%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22mp%22%3A%7B%22min%22%3A1473%2C%22max%22%3A4014%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22tow%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22apco%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A34004%2C%22regionType%22%3A6%7D%5D%2C%22pagination%22%3A%7B%7D%7D",
       "Wallingford":"https://www.zillow.com/wallingford-ct/houses/?searchQueryState=%7B%22usersSearchTerm%22%3A%22Glastonbury%2C%20CT%22%2C%22mapBounds%22%3A%7B%22north%22%3A41.558672631982525%2C%22east%22%3A-72.60943268896483%2C%22south%22%3A41.35437186993799%2C%22west%22%3A-73.01112031103514%7D%2C%22mapZoom%22%3A12%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22price%22%3A%7B%22min%22%3A300000%2C%22max%22%3A650000%7D%2C%22con%22%3A%7B%22value%22%3Afalse%7D%2C%22apa%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22mp%22%3A%7B%22min%22%3A1473%2C%22max%22%3A4014%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22tow%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22apco%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A27747%2C%22regionType%22%3A6%7D%5D%2C%22pagination%22%3A%7B%7D%7D",
       "Avon":"https://www.zillow.com/avon-ct/houses/?searchQueryState=%7B%22usersSearchTerm%22%3A%22Glastonbury%2C%20CT%22%2C%22mapBounds%22%3A%7B%22north%22%3A41.842210290500574%2C%22east%22%3A-72.7562300944824%2C%22south%22%3A41.740588956897106%2C%22west%22%3A-72.95707390551756%7D%2C%22mapZoom%22%3A13%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22price%22%3A%7B%22min%22%3A300000%2C%22max%22%3A650000%7D%2C%22con%22%3A%7B%22value%22%3Afalse%7D%2C%22apa%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22mp%22%3A%7B%22min%22%3A1473%2C%22max%22%3A4014%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22tow%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22apco%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A23560%2C%22regionType%22%3A6%7D%5D%2C%22pagination%22%3A%7B%7D%7D",
       "West Hartford":"https://www.zillow.com/west-hartford-ct/houses/?searchQueryState=%7B%22usersSearchTerm%22%3A%22Glastonbury%2C%20CT%22%2C%22mapBounds%22%3A%7B%22north%22%3A41.817340339282396%2C%22east%22%3A-72.65636809448242%2C%22south%22%3A41.7156795569975%2C%22west%22%3A-72.85721190551757%7D%2C%22mapZoom%22%3A13%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22price%22%3A%7B%22min%22%3A300000%2C%22max%22%3A650000%7D%2C%22con%22%3A%7B%22value%22%3Afalse%7D%2C%22apa%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22mp%22%3A%7B%22min%22%3A1473%2C%22max%22%3A4014%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22tow%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22apco%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A14552%2C%22regionType%22%3A6%7D%5D%2C%22pagination%22%3A%7B%7D%7D",
       "Marlborough":"https://www.zillow.com/marlborough-ct/houses/?searchQueryState=%7B%22usersSearchTerm%22%3A%22Glastonbury%2C%20CT%22%2C%22mapBounds%22%3A%7B%22north%22%3A41.73815160104319%2C%22east%22%3A-72.25580768896484%2C%22south%22%3A41.534418071703875%2C%22west%22%3A-72.65749531103515%7D%2C%22mapZoom%22%3A12%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22price%22%3A%7B%22min%22%3A300000%2C%22max%22%3A650000%7D%2C%22con%22%3A%7B%22value%22%3Afalse%7D%2C%22apa%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22mp%22%3A%7B%22min%22%3A1473%2C%22max%22%3A4014%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22tow%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22apco%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A12575%2C%22regionType%22%3A6%7D%5D%2C%22pagination%22%3A%7B%7D%7D",
       "Farmington":"https://www.zillow.com/farmington-ct/houses/?searchQueryState=%7B%22usersSearchTerm%22%3A%22Glastonbury%2C%20CT%22%2C%22mapBounds%22%3A%7B%22north%22%3A41.78278429452495%2C%22east%22%3A-72.73480259448243%2C%22south%22%3A41.681068731364455%2C%22west%22%3A-72.93564640551759%7D%2C%22mapZoom%22%3A13%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22price%22%3A%7B%22min%22%3A300000%2C%22max%22%3A650000%7D%2C%22con%22%3A%7B%22value%22%3Afalse%7D%2C%22apa%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22mp%22%3A%7B%22min%22%3A1473%2C%22max%22%3A4014%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22tow%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22apco%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A18081%2C%22regionType%22%3A6%7D%5D%2C%22pagination%22%3A%7B%7D%7D"}

def create_links_dataframe():
    df_links = pd.DataFrame.from_dict(data_dict,
                                  columns = ["Link"],
                                  orient ="index")
    return df_links

## Functions
Functions that gather the listing information 

Forked from https://github.com/analyticsariel

In [3]:
def get_zpid(street=None, city=None, state=None, zip_code=None, full_address=None):
    # get search query string
    if full_address == None:
        try:
            query = '{0}, {1}, {2} {3} zillow home details'.format(street, city, state, str(zip_code))
        except:
            return 'Please enter a query string or address details'
    else:
        query = full_address + ' zillow home details'

  # get google search results\
    search_results = search(query, tld='com', lang='en', num=3, start=0, stop=1, pause=0)
    search_results_list = [u for u in search_results]
    url = search_results_list[0] # extract first returned result
  
  # return zpid
    try:
        return [x for x in url.split('/') if 'zpid' in x][0].split('_')[0]
    except:
        return None
     
def get_listings(api_key, listing_url):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/listing"

    querystring = {
        "api_key": api_key,
        "url":listing_url
    }

    return requests.request("GET", url, params=querystring)

def get_property_detail(api_key, zpid):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/property"

    querystring = {
        "api_key": api_key,
        "zpid":zpid
    }

    return requests.request("GET", url, params=querystring)

def get_zpid(api_key, street, city, state, zip_code=None):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/zpidByAddress"

    querystring = {
        "api_key": api_key,
        "street": street,
        "city": city,
        "state": state,
        "zip_code":zip_code
    }

    return requests.request("GET", url, params=querystring)

## Class Execution 

Class Function inwhich the dataframes of each town specified above are combined and organized

Once created the data is exported through Python into a Google Sheet

In [5]:
class CreateHouseData:
    def __init__(self):
        self.df_links    = create_links_dataframe()
        self.df_listings = self.create_listing_dataframe()
        self.df_to_sheet = self.edit_listing_dataframe()
        
    
        
    def create_listing_dataframe(self):
        df_links = self.df_links
        
        df_listings1 = [get_listings(api_key,i) for i in df_links["Link"]]
        df_listings2 = pd.concat([pd.json_normalize(i.json()["data"]["cat1"]["searchResults"]["mapResults"]) for i in df_listings1]).reset_index(drop = True)
        
        return df_listings2
    
    def edit_listing_dataframe(self):
        df_listings = self.df_listings
        
        # Create House Address column
        address_list = [i[len("/homedetails/"):] for i in df_listings["detailUrl"]]
        df_listings["House Address"] = [i[:i.index("/")].replace("-"," ") for i in address_list]
        
        # Create links to Zillow Listing
        df_listings["Listing Link"] = "https://zillow.com" + df_listings["detailUrl"]
        
        # Create Numeric price
        df_listings["NumPrice"] = df_listings["price"].replace('[\$,]', '', regex=True).astype(float)
        
        # Identify which listings are priced below the Zestimate
        df_listings["Price_indicator"] = df_listings["NumPrice"] < df_listings["hdpData.homeInfo.zestimate"]
        
        # Fill all the listings that have no zestimate with N/A
        df_listings["hdpData.homeInfo.zestimate"] = df_listings["hdpData.homeInfo.zestimate"].fillna("N/A").astype(str)
        
        # Change all the Price indicators to N/A where Zestimate is N/A
        df_listings["Price_indicator"] = np.where(df_listings["hdpData.homeInfo.zestimate"]=="N/A","N/A",df_listings["Price_indicator"])
        
        # Formula for generating picture in Google Sheets
        df_listings["imgSrc"] = '=IMAGE("'+df_listings["imgSrc"]+'",4,120,120)'
        
        # Estimate the time listings has been on zillow (recorded in Microseconds converted to days) (rounded up)
        df_listings["timeOnZillow"] = (df_listings["timeOnZillow"]/ 86400000).apply(np.ceil)
        
        # Display lot Size according to unit
        df_listings["Lot Size"] = df_listings['hdpData.homeInfo.lotAreaValue'].round(3).astype(str) +" "+ df_listings["hdpData.homeInfo.lotAreaUnit"]
        
        #Final Clean up to send to Sheet
        df_listings = df_listings.sort_values(by=['timeOnZillow'])
        df_listings = df_listings[["imgSrc","House Address","Listing Link","price","hdpData.homeInfo.zestimate","hdpData.homeInfo.taxAssessedValue","area","beds",'baths','statusText',"timeOnZillow","Lot Size","Price_indicator"]]
        df_listings = df_listings.rename({"Listing Link":"Updated: "+str(pd.Timestamp("today"))+"                 Listing Link","area":"Square Footage","imgSrc":'Picture of House','price':"Price","hdpData.homeInfo.zestimate":"Zestimate","beds":"# of Bedrooms","baths":"# of Bathrooms","statusText":"Market Status","timeOnZillow":'~Days on Zillow',"hdpData.homeInfo.taxAssessedValue":"Taxed Value"}, axis=1)
        
        return df_listings
    
    
    def send_to_sheets(self):
        
        df_to_sheet = self.df_to_sheet
        
        scopes = ['https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive']

        credentials = Credentials.from_service_account_file(r'Enter Service Account for GS Spread', scopes=scopes) # GS spread service account

        gc = gspread.authorize(credentials)

        gauth = GoogleAuth()
        drive = GoogleDrive(gauth)

        # open a google sheet
        gs = gc.open_by_key("1wLGS7fR7CCsEkrLMxjHGstX63GlCWaMqGZWetM66w54") # Sheet Key
        # select a work sheet from its name
        worksheet1 = gs.worksheet('Listing Data') # Sheet

        worksheet1.clear()
        set_with_dataframe(worksheet=worksheet1, dataframe=df_to_sheet, include_index=False,include_column_header=True, resize=True)
        
        return df_to_sheet

    
    
def main(  ):       
    
    df_listings = CreateHouseData().send_to_sheets()
    
    return df_listings


if __name__ == "__main__":
    
    main( )
 

## Conclusion 
A useful tool for people actively looking for houses and properties. This script can be run through a cronjob so that the google spread sheet can be updated on a schedule (hourly, daily, weekly)

Calling on the Zillow API is constrained to 100 calls a month(based on the free subscription at https://www.scrapeak.com/). Each town specified above equates to 1 call on Zillow API so in this function makes 7 calls every time it is run.